# Artificial Nerural Network (ANN)

Vamos a crear una red neuronal que prediga si una cleinte de un banco va cancelar su cuenta en los proximos 6 meses.

### Importamos todo lo que necesitamos

Importamos las librerias necesarias

In [2]:
import numpy as np
import pandas as pd

Importamos el dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')

Checamos el head

In [4]:
dataset.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


Definimos cual es nuestro objetivo y nos deshacemos de las propiedades del dataset que no influyen el resultado

In [6]:
X = dataset.drop(['CustomerId','RowNumber','Surname','Exited'],axis=1)
y = dataset['Exited']

### Preprocesing en X

Transformamos los features que son strings

In [7]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
gender = le.fit_transform(dataset['Gender'])

geo = pd.get_dummies(dataset['Geography'],drop_first=True)

X = X.drop(['Gender','Geography'],axis=1)
X['Gender'] = gender

X = pd.concat([X,geo],axis=1)

Dividimos el dataset para entrenar y probar

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

Standarizamos los dataframes con los features

In [9]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

Checamos el header de los features

In [21]:
X.head(5)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender,Germany,Spain
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1


### Creamos la red neuronal

Importamos las librerias para la red neuronal

In [11]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


Definimos nuestro red neuronal

In [12]:
classifier = Sequential()

Creamos dos hidden layers

In [13]:
classifier.add(Dense(units=6,kernel_initializer='uniform',activation='relu',input_dim=11))
classifier.add(Dense(units=6,kernel_initializer='uniform',activation='relu'))

units: numero de neuronas en el layer ( la suma de los features de X y Y entre dos (X + Y)/2 )

kernel_initializer: la manera en que elige los valores de los weights aleatoriamente

activation: la funcion de activacion

input_dim: el numero de inputs que va tener la red (11 features de X y solo se define en el primer layer)\n

Creamos el output layer

In [14]:
classifier.add(Dense(units=1,kernel_initializer='uniform',activation='sigmoid'))

Compilamos la red neuronal

In [15]:
classifier.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

optmizer = adam es un meotodo de stochastic gradient descent

loss = la funcion de costo y esta vez es es binary_crossentropy porque el ouput es un 0 o 1, si fuera mas de dos seria categorical_crossentropy

matrics = la manera de saber que tan correcto esta el modelo

### Entrenamos la red

In [16]:
classifier.fit(X_train,y_train,batch_size=10,epochs=10,verbose=0)

/Users/manoloesparza/anaconda/envs/python3/lib/python3.4/site-packages/keras/models.py:852: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


### Predicciones

In [17]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [18]:
from sklearn.metrics import classification_report
cr = classification_report(y_test,y_pred)
print(cr)

             precision    recall  f1-score   support

          0       0.84      0.97      0.90      2394
          1       0.73      0.27      0.39       606

avg / total       0.82      0.83      0.80      3000

